# Spot the Fake - CipherCop

## Load and process data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from collections import defaultdict

def clean_and_verify_paths(csv_path, images_folder):
    """
    Reads a CSV, cleans filename entries, and verifies they exist.
    Returns a dictionary mapping brands to valid image paths.
    """
    df = pd.read_csv(csv_path)
    brand_to_images = defaultdict(list)
    found_count = 0
    not_found_count = 0

    # Get a set of all actual filenames for fast lookup
    # os.listdir() is efficient for getting all filenames in a folder
    all_actual_files = set(os.listdir(images_folder))

    for index, row in df.iterrows():
        # Clean the filename from the CSV
        original_filename = str(row['fileName']).strip()  # Remove whitespace
        brand_name = row['logoName']

        # Normalize the filename to lowercase for a case-insensitive check
        normalized_filename = original_filename.lower()

        # Check if the cleaned file exists using a case-insensitive match
        matched_file = None
        for actual_file in all_actual_files:
            if actual_file.lower() == normalized_filename:
                matched_file = actual_file
                break

        if matched_file:
            # Construct the full path and add to the dictionary
            full_path = os.path.join(images_folder, matched_file)
            brand_to_images[brand_name].append(full_path)
            found_count += 1
        else:
            not_found_count += 1
            # You can add a print statement or log here for debugging
            # print(f"Warning: File not found for '{original_filename}'")

    print(f"Verified {found_count} files, skipped {not_found_count} not found.")
    return brand_to_images


In [ ]:
logos_folder_name = '/content/drive/My Drive/AMPBA/CipherCop/Logos/'
csv_file_path = '/content/drive/My Drive/AMPBA/CipherCop/LogoDatabase.csv'
cleaned_brand_dict = clean_and_verify_paths(csv_file_path, logos_folder_name)

Verified 1447 files, skipped 34 not found.


### Process data to create pairs for Siamese network

In [ ]:
import random
def create_siamese_pairs(brand_dict, num_pairs = 500):
  '''
  Set up the data by creating pairs of images. The number of similar pairs
  and the number of dissimilar pairs would be equal to num_pairs.
  So a total of 2 * num_pairs pairs would be created.
  Label will be set to 1 for similar pairs and 0 for dissimilar pairs.
  '''
  all_pairs = []

  # Similar pairs
  brands_with_multiple_images = {brand: images for brand, images in brand_dict.items() if len(images) > 1}
  brand_list = list(brands_with_multiple_images.keys())
  for _ in range(num_pairs):
    brand_name = random.choice(brand_list)
    images = brands_with_multiple_images[brand_name]
    img1_path, img2_path = random.sample(images, 2)
    all_pairs.append({'image1_path': img1_path, 'image2_path': img2_path, 'label': 1})

  # Dissimilar pairs
  brand_list = list(brand_dict.keys())
  for _ in range(num_pairs):
    brand1_name, brand2_name = random.sample(brand_list, 2)
    img1_path = random.choice(brand_dict[brand1_name])
    img2_path = random.choice(brand_dict[brand2_name])
    all_pairs.append({'image1_path': img1_path, 'image2_path': img2_path, 'label': 0})

    return all_pairs

In [ ]:
# Get Siamese pairs and labels
all_pairs = create_siamese_pairs(cleaned_brand_dict, num_pairs = 500)

## Build Siamese network model

In [ ]:
# Create class for Siamese Network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import transforms
from PIL import Image

class SiameseNetwork(nn.Module):
  def __init__(self, base_model):
    super(SiameseNetwork, self).__init__()
    self.base_model = base_model
    # Remove the last classification layer
    # This works for ResNet, not for VGG
    if hasattr(self.base_model, 'fc'):
      self.base_model.fc = nn.Identity()

  def forward(self, input1, input2):
    output1 = self.base_model(input1)
    output2 = self.base_model(input2)
    return output1, output2  # Returns feature embeddings


In [ ]:
# Choose a pre-trained model resnet18
# Experiment with more sophisticated models like resnet50 if time permits
resnet18 = models.resnet18(pretrained=True)

# Create the Siamese network
siamese_net_model = SiameseNetwork(resnet18)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 231MB/s]


### Apply image transformations

In [ ]:
# Transform images for loading into the model
class SiameseDataset(Dataset):
  def __init__(self, all_pairs, transform=None):
    self.all_pairs  = all_pairs
    self.transform = transform

  def __len__(self):
    return len(self.all_pairs)

  def __getitem__(self, idx): # Corrected method name
    pair_dict = self.all_pairs[idx]

    # Unpack from dictionary
    img1_path = pair_dict['image1_path']
    img2_path = pair_dict['image2_path']
    label = pair_dict['label']

    img1 = Image.open(img1_path).convert('RGB')
    img2 = Image.open(img2_path).convert('RGB')

    if self.transform:
      img1 = self.transform(img1)
      img2 = self.transform(img2)

    return img1, img2, torch.tensor(label, dtype=torch.float32)

In [ ]:
# Define transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # Resize images to a fixed size
    transforms.ToTensor(),         # Convert PIL Image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with ImageNet stats
])


### Set up data for loading into model

In [ ]:
# Create the dataset instance
siamese_dataset = SiameseDataset(all_pairs, transform=data_transforms)

# Create the DataLoader instance
batch_size = 32
siamese_dataloader = DataLoader(siamese_dataset, batch_size=batch_size, shuffle=True)

### Set up contrastive loss function, Optimizer

In [ ]:
# Define contrastive loss function
class ContrastiveLoss(nn.Module):
  '''
  Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
  '''
  def __init__(self, margin=2.0):
    super(ContrastiveLoss, self).__init__()
    self.margin = margin

  def forward(self, output1, output2, label):
    euclidean_distance = F.pairwise_distance(output1, output2)

    loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) + \
      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
    return loss_contrastive

In [ ]:
# Initialize the loss function
contrastive_loss = ContrastiveLoss(margin=2.0)

In [ ]:
# Set up optimizer
optimizer = optim.Adam(siamese_net_model.parameters(), lr=0.001)

### Set up device and train model

In [ ]:
# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
siamese_net_model.to(device)

SiameseNetwork(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [ ]:
# Set up the training loop
num_epochs = 10
for epoch in range(num_epochs):
  siamese_net_model.train()
  running_loss = 0.0

  # Iterate over the images in data
  for img1_batch, img2_batch, label_batch in siamese_dataloader:
    img1_batch = img1_batch.to(device)
    img2_batch = img2_batch.to(device)
    label_batch = label_batch.to(device)

    optimizer.zero_grad()

    output1, output2 = siamese_net_model(img1_batch, img2_batch)

    loss = contrastive_loss(output1, output2, label_batch)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  epoch_loss = running_loss / len(siamese_dataloader)
  print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


Epoch 1/10, Loss: 0.8692
Epoch 2/10, Loss: 0.3970
Epoch 3/10, Loss: 0.0971
Epoch 4/10, Loss: 0.0232
Epoch 5/10, Loss: 0.0627
Epoch 6/10, Loss: 0.1223
Epoch 7/10, Loss: 0.0642
Epoch 8/10, Loss: 1.9810
Epoch 9/10, Loss: 0.1248
Epoch 10/10, Loss: 0.1138


In [ ]:
# Save the model
torch.save(siamese_net_model.state_dict(), 'siamese_net_model.pth')

# Model full
#torch.save('siamese_net_model_full.pth')

## Implement Explainable AI

### Grad-CAM

In [ ]:
import matplotlib.pyplot as plt

# Set the model to evaluation mode
siamese_net_model.eval()

# Choose an image pair for demonstration (using the same pair as Saliency Maps)
img1, img2, label = siamese_dataset[0]

# Move the images to the device and add a batch dimension
img1 = img1.to(device).unsqueeze(0)
img2 = img2.to(device).unsqueeze(0)

# We need to access the feature maps from a convolutional layer
# For ResNet18, a good layer to target is the last convolutional block before the fully connected layer
target_layer = siamese_net_model.base_model.layer4[-1]

# Clear any existing hooks on the target layer
# This is important if the cell has been run multiple times
if hasattr(target_layer, '_forward_hooks'):
    target_layer._forward_hooks.clear()
if hasattr(target_layer, '_backward_hooks'):
    target_layer._backward_hooks.clear()
if hasattr(target_layer, '_full_backward_hooks'):
    target_layer._full_backward_hooks.clear()


# Store gradients and activations
gradients = None
activations = None

def save_gradients(module, grad_input, grad_output):
    global gradients
    # We are interested in the gradient of the output of the layer
    gradients = grad_output[0]

def save_activations(module, input, output):
    global activations
    activations = output

# Register hooks to the target layer
hook_grad = target_layer.register_full_backward_hook(save_gradients)
hook_act = target_layer.register_forward_hook(save_activations)

# Forward pass through the model
output1, output2 = siamese_net_model(img1, img2)

# Calculate the distance between the embeddings
euclidean_distance = torch.nn.functional.pairwise_distance(output1, output2)

# Backpropagate the gradient of the distance with respect to the output
# We'll backpropagate a scalar '1' to get the gradients of the distance
euclidean_distance.backward(torch.ones_like(euclidean_distance))

# Remove hooks
hook_grad.remove()
hook_act.remove()

# Get the weights for Grad-CAM by global average pooling the gradients
# Gradients have shape (batch_size, channels, height, width)
# Average across height and width dimensions
pooled_gradients = torch.mean(gradients, dim=[2, 3])

# Weight the channels of the activation maps by the pooled gradients
# Activations have shape (batch_size, channels, height, width)
for i in range(activations.shape[0]):
    for j in range(pooled_gradients.shape[1]):
        activations[i, j, :, :] *= pooled_gradients[i, j]

# Sum the weighted activation maps across channels to get the heatmap
heatmap = torch.sum(activations, dim=1).squeeze(0) # Remove batch and channel dimensions

# Apply ReLU to the heatmap
heatmap = F.relu(heatmap)

# Normalize the heatmap
# Add a small epsilon to avoid division by zero if max is 0
heatmap = heatmap / (torch.max(heatmap) + 1e-8)

# Upsample the heatmap to the original image size
# Need to reverse the transforms for visualization
# We can resize the heatmap to the size of the input image before transforms (224, 224)
heatmap = F.interpolate(heatmap.unsqueeze(0).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False)
heatmap = heatmap.squeeze().detach().cpu().numpy()

# Convert the original image tensor back to PIL Image for visualization
img1_display = img1.squeeze(0).detach().cpu()
mean = torch.tensor([0.485, 0.456, 0.406]).view(-1, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(-1, 1, 1)
img1_display = img1_display * std + mean
img1_display = torch.clamp(img1_display, 0, 1)
img1_display = transforms.ToPILImage()(img1_display)


# Visualize the original image and the Grad-CAM heatmap
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(img1_display)
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img1_display) # Display original image
plt.imshow(heatmap, cmap='jet', alpha=0.5) # Overlay heatmap
plt.title("Grad-CAM Heatmap")
plt.axis('off')

plt.tight_layout()
plt.show()

print("Grad-CAM heatmap generated for the first image of the pair.")

### Saliency maps

In [ ]:
# Set the model to evaluation mode
siamese_net_model.eval()

# Choose an image pair from the dataset for demonstration
# We'll use the first pair from the optimized dataset
img1, img2, label = siamese_dataset[0]

# Move the images to the device
img1 = img1.to(device)
img2 = img2.to(device)

# We need to calculate gradients with respect to the input image, so we set requires_grad to True
img1.requires_grad_(True)
img2.requires_grad_(True)


# Forward pass through the model
output1, output2 = siamese_net_model(img1.unsqueeze(0), img2.unsqueeze(0)) # Add batch dimension

# Calculate the distance between the embeddings
euclidean_distance = torch.nn.functional.pairwise_distance(output1, output2)

# To get the saliency map for img1 with respect to the distance,
# we need to backpropagate the gradient of the distance to img1.
# We can treat the distance as a scalar output for this purpose.
euclidean_distance.backward()

# Get the gradient with respect to img1
saliency_map = img1.grad.data.abs().mean(dim=0) # Take the mean across color channels

# Normalize the saliency map for better visualization
saliency_map = saliency_map / saliency_map.max()

# Convert the image tensor back to PIL Image for visualization
# We need to reverse the normalization and permutations
img1_display = img1.detach().cpu()
# Reverse normalization (using mean and std used in transforms)
mean = torch.tensor([0.485, 0.456, 0.406]).view(-1, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(-1, 1, 1)
img1_display = img1_display * std + mean
img1_display = torch.clamp(img1_display, 0, 1) # Clamp values to be within [0, 1]
img1_display = transforms.ToPILImage()(img1_display)


# Visualize the original image and the saliency map
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(img1_display)
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(saliency_map.cpu(), cmap='hot')
plt.title("Saliency Map")
plt.axis('off')

plt.tight_layout()
plt.show()

print("Saliency Map generated for the first image of the pair.")